In [1]:
import torch

# layer 비교하기 traffic 데이터를 가져오는 식으로 비교
from layers.Embed import *


In [2]:
# 관련 패키지 불러오기

import torch.nn as nn
import torch.nn.functional as F
from layers.Transformer_EncDec import Encoder, EncoderLayer
from layers.SelfAttention_Family import FullAttention, AttentionLayer
import numpy as np

In [3]:
# 변수 정리

seq_len = 96 # c_in으로 
label_len = 48
pred_len = 48
e_layers = 2
d_layers = 1
factor = 3
enc_in = 321
dec_in = 321
d_model = 512
batch_size = 32
c_out = 321
des = 'Exp'
learning_rate = 0.001
freq = 'h' # 입력빈도 - 세팅을 일일이 해야 하는 부분
dropout = 0.1
embed = 'timeF' # timeF, fixed, learned
class_strategy = 'projection'

In [4]:
enc_transformer_embedding_0 = DataEmbedding(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [5]:
enc_transformer_embedding_0

DataEmbedding(
  (value_embedding): TokenEmbedding(
    (tokenConv): Conv1d(96, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (position_embedding): PositionalEmbedding()
  (temporal_embedding): TimeFeatureEmbedding(
    (embed): Linear(in_features=4, out_features=512, bias=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

In [6]:
enc_transformer_embedding_1 = DataEmbedding_inverted(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [7]:
enc_transformer_embedding_1

DataEmbedding_inverted(
  (value_embedding): Linear(in_features=96, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [8]:
enc_transformer_embedding_2 = DataEmbedding_inverted_TCN(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [9]:
enc_transformer_embedding_2

DataEmbedding_inverted_TCN(
  (value_embedding): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (net): Sequential(
          (0): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): ReLU()
          (2): Dropout(p=0.2, inplace=False)
          (3): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (4): ReLU()
          (5): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(96, 512, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(2,), dilation=(2,))
        (relu1): ReLU()
        

In [11]:
dummy_input = torch.randn(batch_size, enc_in, seq_len)

In [12]:
dummy_input


tensor([[[ 1.4471, -0.1383, -0.1616,  ..., -0.0615, -1.1846,  1.1072],
         [ 0.0789, -0.9835, -0.5236,  ..., -0.6217, -1.0600,  0.4784],
         [ 0.1917,  0.3444, -1.2287,  ..., -0.6268, -0.3823,  0.7940],
         ...,
         [ 0.6642, -0.1015,  0.0799,  ...,  0.3088,  1.3785, -0.1309],
         [ 0.9941,  1.3404,  0.8640,  ...,  0.3308, -0.6234, -2.2929],
         [-0.8761,  0.6225,  0.7319,  ..., -0.0788, -0.5455, -1.0578]],

        [[ 0.2641, -0.8465, -0.1363,  ..., -1.6826, -1.0207,  0.6776],
         [-0.9248, -2.3743,  0.8127,  ..., -0.4400, -1.6505,  0.7345],
         [-0.8364, -0.5805, -0.8539,  ..., -1.2674, -0.8364, -0.2391],
         ...,
         [ 1.1862,  0.5631,  0.2783,  ...,  0.3763, -0.4193, -2.0210],
         [-1.1822,  0.3717,  1.9572,  ...,  1.0209, -1.3553,  0.3346],
         [-0.0609,  0.1561,  0.2156,  ...,  0.7570,  0.2979, -0.7982]],

        [[ 0.9947, -0.7507,  1.2363,  ..., -1.1301,  2.0397,  0.8976],
         [-1.1290,  1.1042,  0.6116,  ..., -1

In [13]:
dummy_input.size()

torch.Size([32, 321, 96])

In [14]:
dummy_input_0 = dummy_input.permute(0,2,1)

In [15]:
output_0 = enc_transformer_embedding_0(dummy_input, None)

In [16]:
output_0.size()

torch.Size([32, 321, 512])

In [18]:
output_1 = enc_transformer_embedding_1(dummy_input_0, None)

In [19]:
output_1

tensor([[[-3.3098e-01,  6.3905e-01, -5.8549e-01,  ..., -6.5899e-01,
           4.5427e-02, -2.9960e-01],
         [ 4.5091e-01,  9.7193e-02,  4.3653e-01,  ..., -1.0514e+00,
           3.7163e-01, -2.0646e-01],
         [-5.0228e-01, -2.6824e-01, -1.1385e-01,  ...,  8.2904e-02,
           3.0099e-01, -8.3608e-01],
         ...,
         [ 6.5376e-01,  7.3982e-01, -0.0000e+00,  ...,  6.5805e-02,
           5.2746e-01, -5.9305e-01],
         [ 1.4339e+00,  5.9361e-01,  8.8020e-01,  ..., -1.3848e-01,
           2.5996e-02,  2.5043e-01],
         [ 0.0000e+00,  3.8074e-01,  5.4075e-01,  ..., -2.9867e-01,
           4.8203e-01, -9.3896e-01]],

        [[ 5.2140e-02,  1.9072e-01, -0.0000e+00,  ..., -6.4225e-01,
           7.1435e-01, -3.4063e-01],
         [ 1.4512e-01,  1.5817e-01,  8.9681e-02,  ..., -2.1585e-02,
           3.5318e-01, -1.5188e+00],
         [-0.0000e+00, -6.1943e-01, -2.5924e-01,  ...,  5.9583e-01,
          -3.7397e-01, -1.0606e+00],
         ...,
         [ 1.3554e+00,  9

In [20]:
output_1.size()

torch.Size([32, 321, 512])

In [21]:
output_2 = enc_transformer_embedding_2(dummy_input_0, None)

Input size: torch.Size([32, 96, 321])
Input size: torch.Size([32, 512, 323])
Input size: torch.Size([32, 512, 327])
torch.Size([32, 335, 512])


In [37]:
temporal_block = TemporalBlock(96, 512, 2,1,1,1)

In [39]:
output_temporal = temporal_block(dummy_input_0)

In [40]:
output_temporal.size()

torch.Size([32, 512, 323])

In [38]:
output_temporal = temporal_block(dummy_input)

RuntimeError: Given groups=1, weight of size [512, 96, 2], expected input[32, 321, 96] to have 96 channels, but got 321 channels instead

In [73]:
output_temporal

tensor([[[2.9887e-02, 1.2409e-01, 1.7761e-01,  ..., 6.5624e-02,
          9.7772e-03, 0.0000e+00],
         [0.0000e+00, 2.5196e-01, 1.3163e-02,  ..., 4.3740e-02,
          1.2417e-02, 2.1450e-02],
         [1.3100e-01, 1.3373e-01, 5.9980e-02,  ..., 0.0000e+00,
          7.0686e-03, 0.0000e+00],
         ...,
         [7.1469e-02, 0.0000e+00, 8.3666e-02,  ..., 3.7311e-03,
          1.3744e-02, 3.9985e-03],
         [0.0000e+00, 0.0000e+00, 1.4286e-02,  ..., 1.8574e-01,
          3.7800e-02, 0.0000e+00],
         [1.3890e-01, 8.2674e-02, 1.6861e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 2.3342e-01, 1.2673e-01,  ..., 1.3788e-01,
          0.0000e+00, 7.7979e-03],
         [9.6090e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          8.7233e-03, 5.6930e-02],
         [1.1677e-01, 0.0000e+00, 5.1605e-02,  ..., 3.8304e-02,
          2.1290e-02, 0.0000e+00],
         ...,
         [1.2013e-01, 2.0939e-01, 8.7630e-02,  ..., 0.0000e+00,
          9.215

In [93]:
output_temporal.size()

torch.Size([32, 512, 98])

In [33]:
temporal_conv_net = TemporalConvNet(96, [512, 512, 512])

In [35]:
output_temporal_conv_net = temporal_conv_net(dummy_input_0)

In [77]:
output_temporal_conv_net

tensor([[[0.0249, 0.0278, 0.0253,  ..., 0.0000, 0.0185, 0.0000],
         [0.0203, 0.0000, 0.1856,  ..., 0.0029, 0.0084, 0.0009],
         [0.0361, 0.1856, 0.0498,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0266, 0.0214, 0.1779,  ..., 0.0000, 0.0102, 0.0115],
         [0.1257, 0.0316, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.1381, 0.0740, 0.1175,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1560, 0.0338, 0.0129,  ..., 0.0168, 0.0184, 0.0190],
         [0.0000, 0.0776, 0.2084,  ..., 0.0044, 0.0059, 0.0027],
         [0.0862, 0.0733, 0.1529,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0796, 0.1283, 0.2700,  ..., 0.0071, 0.0089, 0.0077],
         [0.0000, 0.0000, 0.0619,  ..., 0.0000, 0.0000, 0.0000],
         [0.0028, 0.0601, 0.1972,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1644, 0.0188, 0.0992,  ..., 0.0239, 0.0170, 0.0170],
         [0.1283, 0.0000, 0.0668,  ..., 0.0013, 0.0049, 0.0030],
         [0.3076, 0.1029, 0.0309,  ..., 0.0000, 0.0000, 0.

In [36]:
output_temporal_conv_net.size()

torch.Size([32, 512, 335])